In [1]:
import sqlite3 as sql

In [11]:
import datetime

In [1212]:
from abc import ABC

In [2]:
database = "library.db"
connection = sql.connect(database)

In [3]:
connectDB = connection.cursor()

In [4]:
connectDB.execute("""CREATE table Authors (
                    id integer not null primary key,
                    name text)""")

connectDB.execute("""CREATE table Publications (
                    id integer not null primary key,
                    name text)""")

connectDB.execute("""CREATE table Books (
                    id integer not null primary key,
                    name text,
                    copy text,
                    genre text,
                    authID integer,
                    pubID integer,
                    FOREIGN KEY (authID) REFERENCES Authors (id) ON DELETE CASCADE ON UPDATE NO ACTION,
                    FOREIGN KEY (pubID) REFERENCES Publications (id) ON DELETE CASCADE ON UPDATE NO ACTION)""")

In [74]:
connectDB.execute("""CREATE table Members(
                    id integer not null primary key,
                    name text,
                    status text)""")

In [75]:
connectDB.execute("""CREATE table Issue(
                    id integer,
                    bookID integer,
                    issuerID integer,
                    timeofIsuue datetime,
                    returnDate datetime,
                    status text,
                    FOREIGN KEY (issuerID) REFERENCES Members (id) ON DELETE CASCADE ON UPDATE NO ACTION
                    )""")

In [1207]:
class Tester:
    
    def searchByBookName(self, bookName):
        sBook=SearchBook()
        return sBook.search(bookName)
        
    def searchByAuthor(self, authorName):
        sAuthor=SearchAuthor()
        return sAuthor.search(authorName)
    
    def searchByPublication(self, publishingHouse):
        sPub=SearchPublication()
        return sPub.search(publishingHouse)
    
    def searchByGenre(self, genre):
        sGenre = SearchGenre()
        sGenre.search(genre)
    
    def searchMemberName(self, name):
        sName=SearchMemberByName()
        return sName.search(name)
        
    
    def lastBookAdded(self):
        countOfBooks=connectDB.execute("""Select count(*) from Books""")
        countOfBooks=countOfBooks.fetchone()[0]
        if countOfBooks == 0:
            return 0
        lastBook=connectDB.execute("Select * from Books")
        for last in lastBook:
            lastBk=last[0]
        
        return lastBk
    
    def addBook(self, bookName, numberOfCopies, genre, author, publication):
        bkName, bkId = self.searchByBookName(bookName)
        if bkId == -1:
            countOfBooks=self.lastBookAdded()
            countOfBooks+=1
            authName, authId = self.searchByAuthor(author)
            if authId == -1:
                authName, authId = self.addAuthor(author)
            pubName, pubId = self.searchByPublication(publication)
            if pubId == -1:
                pubName, pubId = self.addPublication(publication)
            connectDB.execute("INSERT into Books values(:id, :name, :copy, :genre, :authID, :pubID)",\
                              {'id':countOfBooks, 
                               'name':bookName,
                               'copy':numberOfCopies,
                               'genre':genre,
                               'authID':authId,
                               'pubID':pubId})
            connection.commit()
            return f'{bookName} added to the database'
        return "Book Present in the Database"
    
    def addAuthor(self, authorName):
        authName, authorId = self.searchByAuthor(authorName)
        if authorId == -1:
            countOfAuthors=connectDB.execute("""Select count(*) from Authors""")
            countOfAuthors=countOfAuthors.fetchone()[0]
            countOfAuthors+=1
            connectDB.execute("INSERT into Authors values(:id, :name)", {'id':countOfAuthors, 'name':authorName})
            connection.commit()
            return authorName, countOfAuthors
        return authName, authorId
            
    def addPublication(self, publicationName):
        pubName, pubId = self.searchByPublication(publicationName)
        if pubId == -1:
            countOfPublications=connectDB.execute("""Select count(*) from Publications""")
            countOfPublications=countOfPublications.fetchone()[0]
            countOfPublications+=1
            connectDB.execute("INSERT into Publications values(:id, :name)", {'id':countOfPublications, 'name':publicationName})
            connection.commit()
            return publicationName, countOfPublications
        return pubName, pubId
    
    def deleteBook(self, bookName):
        bookName = connectDB.execute("Select * from Books where name=?", (bookName,))
        bookName = bookName.fetchall()
        if not len(bookName):
            return "Book not present in the database"
        else:
            connectDB.execute("Delete from Books where name=?",(bookName[0][1],))
            connection.commit()
            return f'Book {bookName[0][1]} has been deleted'
    
    def addMember(self, memberName, status):
        countMembers = connectDB.execute("Select count(*) from Members")
        countMembers = countMembers.fetchone()[0]
        countMembers+=1
        
        connectDB.execute("INSERT into Members values(:id, :name, :status)",{'id':countMembers, 'name':memberName, 'status':'Active'})
        connection.commit()
        return f'New Library Member {memberName} added'
    
    def changeMemberStatus(self, memberName):
        memberStatus = connectDB.execute("SELECT status, id from Members where name=?",(memberName,))
        memberStatusHelper = memberStatus.fetchone()
        memberStat = memberStatusHelper[0]
        memberID = memberStatusHelper[1]
        if memberStat == 'Blocked':
            memberStat = 'Active'
        else:
            memberStat = 'Blocked'
        connectDB.execute("UPDATE Members SET status=:memStat where id=:memID",{'memStat':memberStatus, 'memID':memberID,})
        connection.commit()
        return f"{memberName}'s status has been updated to {memberStatus}"
    
    def issueBook(self, bookName, issuerName):
        bookDetails = connectDB.execute("SELECT copy, id from Books where name=?", (bookName,))
        bookDetails = bookDetails.fetchone()
        bookCount, bookId = bookDetails[0], bookDetails[1]
        issueMember = connectDB.execute("SELECT id, status from Members where name=?", (issuerName,))
        issueMembers = issueMember.fetchone()
        issueMemberId, memberStatus = issueMembers[0], issueMembers[1]
        booksByMember=self.calculateBooksIssuedByMember(issuerName)
        memberFine = self.calculateFine(issuerName)
        if int(bookCount)>0 and memberStatus!='Blocked' and booksByMember<5 and memberFine==0:
            issueCount = connectDB.execute("SELECT count(*) from Issue")
            issueCount = issueCount.fetchone()[0]
            issueCount+=1
            connectDB.execute("INSERT into Issue values(:id, :bookId, :issuerId, :timeOfIsuue, :returnDate, :status)", \
                             {'id':issueCount,
                              'bookId':bookId,
                              'issuerId':issueMemberId,
                              'timeOfIsuue':datetime.datetime.now(),
                              'returnDate':None,
                              'status':'Not Returned'})
            connectDB.execute("UPDATE Books SET copy=? WHERE id=?",(int(bookCount)-1, bookId,))
            connection.commit()
            return f'{bookName} was issued to {issuerName} for 30 days'
        return f'Cannot Issue {bookName} to {issuerName}'
        
    def returnBook(self, memberName, bookName):
        memberID = connectDB.execute("SELECT id from Members where name=?",(memberName,))
        memberID = memberID.fetchone()[0]
        bookID = connectDB.execute("SELECT id from Books where name=?",(bookName,))
        bookID = bookID.fetchone()[0]
        issueID = connectDB.execute("SELECT id from Issue where Issue.issuerID=? and Issue.bookID=? and Issue.status=?",(memberID, bookID, 'Not Returned',))
        issueID = issueID.fetchone()[0]
        connectDB.execute("UPDATE Issue SET status=:newStatus, returnDate=:retDate where id=:id",{'newStatus':'Returned','retDate':datetime.datetime.now(),'id':issueID})
        copies=connectDB.execute("SELECT copy from Books where id=?",(bookID,))
        copies=copies.fetchone()[0]
        copies=int(copies)
        copies+=1
        connectDB.execute("UPDATE Books SET copy=:copies where id=:bookID",{'copies':copies, 'bookID':bookID})
        connection.commit()
        return f"{bookName} has been returned by {memberName}"
        
    def getBookBorrower(self, bookName):
        bookID=connectDB.execute("SELECT id from Books where name=?",(bookName,))
        bookID = bookID.fetchone()[0]
        records=connectDB.execute("SELECT Members.name, Issue.timeofIsuue from Members INNER JOIN Issue ON Members.id=Issue.IssuerID where Issue.bookID=? and Issue.status=?",(bookID,'Not Returned',))
        records = records.fetchall()
        if not len(records):
            print(f'{bookName} was issued by None')
        else:
            print(f'{bookName} issued by the following Members')
            print("Member Name", "\t", "Date and Time")

            for record in records:
                print(record[0][:20], end="\t\t")
                print(record[1][:-10])
        
    def getMemberBookDetails(self, memberName):
        memberID = connectDB.execute("SELECT id from Members where name=?",(memberName,))
        memberID = memberID.fetchone()[0]
        memberBooks = connectDB.execute("SELECT name from Books INNER JOIN Issue ON Issue.bookID=Books.id where Issue.IssuerID=? and Issue.status=?",(memberID,'Not Returned',))
        memberBooks = memberBooks.fetchall()
        
        if not len(memberBooks):
            print(f"No Books issued by {memberName}")
            return 0
        
        print(f"Books issued by {memberName}")
        for memberBook in memberBooks:
            print(memberBook[0])
        return len(memberBooks)
        
    def calculateFine(self, memberName):
        memberID = connectDB.execute("SELECT id from Members where name=?",(memberName,))
        memberID = memberID.fetchone()[0]
        memberBooks = connectDB.execute("SELECT Books.name, Issue.timeofIsuue, Issue.returnDate from Books INNER JOIN Issue ON Issue.bookID=Books.id where Issue.IssuerID=?",(memberID,))
        memberBooks = memberBooks.fetchall()
        if not len(memberBooks):
            return 0
        
        print(f"Fine Sheet for {memberName}: ")
        fine=0
        print("Book", "\t\t\t", "Issue Date", "\t", "Return Date", "\t", "Fine")
        for memberBook in memberBooks:
            issueDate = memberBook[1]
            returnDate = memberBook[2]
            if returnDate is None:
                fine+=0
            else:
                format = '%Y-%m-%d'
                issueDate = datetime.datetime.strptime(issueDate[:10], format)
                returnDate = datetime.datetime.strptime(returnDate[:10], format)
                day = returnDate - issueDate
                if day.days>30:
                    extraDays = day.days-30
                    fine+=(extraDays*10)
            print(memberBook[0][:20], end="\t\t")
            print(memberBook[1][:10], end="\t")
            if memberBook[2]:
                print(memberBook[2][:10], end="\t\t ")
            else:
                print(memberBook[2], end="\t\t ")
            print(fine)
        
        print("Total Fine is: ",fine)
        return fine
        
    
    def calculateBooksIssuedByMember(self, memberName):
        return self.getMemberBookDetails(memberName)

In [1208]:
test=Tester()

In [885]:
class BookSearch(ABC):
    def search(self):
        pass

In [835]:
class SearchBook(BookSearch):
    def search(self, bookName):
        books = connectDB.execute("Select * from Books where name=?", (bookName,))
        books = books.fetchall()
        if not len(books):
            return "Books not available", -1
        for book in books:
            return book[1], book[0]

In [836]:
class SearchAuthor(BookSearch):
    def search(self, authorName):
        authors = connectDB.execute("Select * from Authors where name=?", (authorName,))
        authors = authors.fetchall()
        if not len(authors):
            return "Author not Available", -1
        for author in authors:
            return author[1], author[0]

In [837]:
class SearchPublication(BookSearch):
    def search(self, publishingHouse):
        publications = connectDB.execute("Select * from Publications where name=?", (publishingHouse,))
        publications = publications.fetchall()
        if not len(publications):
            return "Publication not Available", -1
        for publication in publications:
            return publication[1], publication[0]

In [838]:
class searchGenre(BookSearch):
    def search(self, genre):
        genres = connectDB.execute("Select name from Books where genre=?", (genre,))
        if not len(genres.fetchall()):
            return "Genre not Available"
        for genre in genres:
            print(genre)

In [839]:
class MemberSearch(ABC):
    def search(self):
        pass

In [840]:
class SearchMemberByID(MemberSearch):
    def search(self, memberID):
        member=connectDB.execute("SELECT * from Members where id=?",(memberID,))
        return member.fetchone()[0]

In [841]:
class SearchMemberByName(MemberSearch):
    def search(self, memberName):
        member = connectDB.execute("SELECT * from Members where name=?",(memberName,))
        return member.fetchone()[0]

In [1209]:
test.addBook("Kafka On The Shore", 2, "Novel", "Murukami", "McMilan Books")

'Kafka On The Shore added to the database'

In [1170]:
test.addBook("IELTS Test Prep", 1, "Education", "Dr Kiran Preet Kaur", "British Council")

'IELTS Test Prep added to the database'

In [1169]:
test.deleteBook('IELTS Test Prep')

'Book IELTS Test Prep has been deleted'

In [796]:
test.addMember('Glen','Active')

'New Library Member Glen added'

In [798]:
test.addMember('Vinisha', 'Active')

'New Library Member Vinisha added'

In [799]:
test.addMember('Cristiano', 'Active')

'New Library Member Cristiano added'

In [801]:
test.addMember('Elvisha', 'Active')

'New Library Member Elvisha added'

In [901]:
test.issueBook('Rich Dad Poor Dad', 'Elvisha')

'Rich Dad Poor Dad was issued to Elvisha for 30 days'

In [1171]:
test.issueBook('IELTS Test Prep', 'Vinisha')

Books issued by Vinisha
Rich Dad Poor Dad


'IELTS Test Prep was issued to Vinisha for 30 days'

In [1141]:
test.getBookBorrower('Rich Dad Poor Dad')

Rich Dad Poor Dad issued by the following Members
Member Name 	 Date and Time
Vinisha		2022-05-14 10:32
Glen		2022-05-14 10:32
Elvisha		2022-05-14 10:32


In [1151]:
test.getMemberBookDetails('Vinisha')

Books issued by Vinisha
Rich Dad Poor Dad
IELTS Test Prep


2

In [1206]:
test.calculateFine('Vinisha')

Fine Sheet for Vinisha: 
Book 			 Issue Date 	 Return Date 	 Fine
Rich Dad Poor Dad		2022-05-14	None		 0
IELTS Test Prep		2022-05-14	2022-05-14		 0
Total Fine is:  0


0

In [1172]:
test.returnBook('Vinisha', 'IELTS Test Prep')

'IELTS Test Prep has been returned by Vinisha'

In [987]:
bookName = 
bookID=connectDB.execute("SELECT id from Books where name=?",(bookName,))
bookID = bookID.fetchone()[0]
records=connectDB.execute("SELECT Members.name, Issue.timeofIsuue from Members INNER JOIN Issue ON Members.id=Issue.IssuerID where Issue.bookID=?",(bookID,))

In [1211]:
test.getMemberBookDetails('Glen')

Books issued by Glen
Rich Dad Poor Dad
Kafka On The Shore


2

In [1173]:
x=connectDB.execute("SELECT * from Issue")
x=x.fetchall()

In [1163]:
x=connectDB.execute("DELETE from Issue where id=?",(6,))
connection.commit()

In [1210]:
test.issueBook('Kafka On The Shore','Glen')

Books issued by Glen
Rich Dad Poor Dad
Fine Sheet for Glen: 
Book 			 Issue Date 	 Return Date 	 Fine
Rich Dad Poor Dad		2022-05-14	None		 0
Total Fine is:  0


'Kafka On The Shore was issued to Glen for 30 days'

In [1186]:
isDate = '2022-05-14'
format = '%Y-%m-%d'
isDT = datetime.datetime.strptime(isDate, format)

In [1193]:
retdate = '2022-06-24'
rtDT = datetime.datetime.strptime(retdate, format)

In [1196]:
d=rtDT-isDT

In [1197]:
d.days

41